# Initial Setup

In [ ]:
from bs4 import BeautifulSoup
import os
from datetime import datetime
from natsort import os_sorted

In [ ]:
# Create a file list sorted by first to last message

folder = "c:/Users/DANKINAH/Downloads/Personal/Messages/" #Daniel/Documents/Archive/Facebook Data/messages/inbox/FREENEXTIDE_EbQwaw_iLQ/"
file_list = []
message_set = set()
message_list = []

for filename in os.listdir(folder):
    if filename.endswith('.html'):
       fname = os.path.join(folder, filename)
       file_list.append(fname)

file_list = os_sorted(file_list)
file_list.reverse()
print(file_list)

In [ ]:
# Grab messages from files and place in a sorted list

for filename in file_list:
	with open(filename, encoding='utf-8') as f:
		soup = BeautifulSoup(f, 'html.parser')
	cur_message_list = []
	messages = soup.find_all(class_='pam _3-95 _2pi0 _2lej uiBoxWhite noborder')
	for message in messages[1:]:
		sender = message.contents[0].string
		content = message.contents[1].contents[0].contents[1].string
		timestamp = datetime.strptime(str(message.contents[2].string), '%d %b %Y, %H:%M')
		message_tuple = (sender,content,timestamp)
		cur_message_list.append(message_tuple)
	cur_message_list.reverse()
	message_list += cur_message_list
	print(filename, len(message_list))

# Run each of these individually

In [ ]:
#Print nicknames for each member

nick = ['set the nickname for', 'set his own nickname', 'set your nickname']

nicknames = dict()

for message_tuple in message_list:
	content = message_tuple[1]
	if not content or 'nickname' not in content:
		continue
	if nick[0] in content:
		nickname = content.split(nick[0])[1].split(' to ')[1][:-1]
		name = content.split(nick[0])[1].split(' to ')[0][1:]
	elif nick[1] in content or nick[2] in content:
		nickname = content.split('nickname')[1].split(' to ')[1][:-1]
		name = message_tuple[0]

	if name not in nicknames:
		nicknames[name] = []
	else:
		nicknames[name].append(nickname)

print('Nicknames of each member')
for name in nicknames.keys():
    display(name + ": " + ', '.join(nicknames[name]) + "\n")

In [ ]:
#Print nicknames for each member grouped by day

import pandas as pd

nick = ['set the nickname for', 'set his own nickname', 'set your nickname']

nicknames = []

for message_tuple in message_list:
	content = message_tuple[1]
	if not content or 'nickname' not in content:
		continue
	if nick[0] in content:
		nickname = content.split(nick[0])[1].split(' to ')[1][:-1]
		name = content.split(nick[0])[1].split(' to ')[0][1:]
	elif nick[1] in content or nick[2] in content:
		nickname = content.split('nickname')[1].split(' to ')[1][:-1]
		name = message_tuple[0]

	nicknames.append({'name': name, 'nickname': nickname, 'datetime': message_tuple[2]})

df = pd.DataFrame(nicknames)
df = df.groupby(df.datetime.dt.date)['nickname'].apply(', '.join).to_frame()

#df.style.set_properties(**{'text-align': 'left','white-space': 'pre-wrap'})

print('Nicknames grouped by day')
display(HTL.to_html())

In [ ]:
#Print amount each member has set a nickname

nick = ['set the nickname for', 'set his own nickname', 'set your nickname']

members = dict()

for message_tuple in message_list:
	content = message_tuple[1]
	if not content:
		continue
	if any([x in content for x in nick]):
		name = message_tuple[0]
		if name not in members:
			members[name] = 0
		else:
			members[name] += 1

print('Amount each member has set a nickname')
display(members)

In [ ]:
#Print amount each member has said a certain word

word = 'friend'

members = dict()

for message_tuple in message_list:
	content = message_tuple[1]
	if not content:
		continue
	if word in content:
		name = message_tuple[0]
		if name not in members:
			members[name] = 0
		else:
			members[name] += 1
print('Word frequency of: ' + word)
display(members)